In [1]:
import numpy as np
import pandas as pd
import math
import requests
from scipy.stats import percentileofscore as score
import xlsxwriter

In [2]:
from secret import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.tail()

,Ticker
496,YUM
497,ZBH
498,ZBRA
499,ZION
500,ZTS


In [3]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2602824558638,
 'week52high': 185.91,
 'week52low': 121.63,
 'week52highSplitAdjustOnly': 189.09,
 'week52lowSplitAdjustOnly': 125.38,
 'week52change': 0.2151562638217713,
 'sharesOutstanding': 16675887728,
 'float': 0,
 'avg10Volume': 113513271,
 'avg30Volume': 95999455,
 'day200MovingAvg': 166.25,
 'day50MovingAvg': 171.47,
 'employees': 148197,
 'ttmEPS': 6.19,
 'ttmDividendRate': 0.8995362832416299,
 'dividendYield': 0.005810084941597394,
 'nextDividendDate': '',
 'exDividendDate': '2022-05-01',
 'nextEarningsDate': '2022-07-14',
 'peRatio': 25.852874233842666,
 'beta': 0,
 'maxChangePercent': 59.93129745824977,
 'year5ChangePercent': 3.376113516419555,
 'year2ChangePercent': 1.0795393913646112,
 'year1ChangePercent': 0.2187086430629641,
 'ytdChangePercent': -0.1173062847172158,
 'month6ChangePercent': 0.04898053216370363,
 'month3ChangePercent': -0.0994832651302273,
 'month1ChangePercent': -0.075037228848915,
 'day30ChangePercent': -0.075

In [4]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

df_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [5]:
print(symbol_strings)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS', 'GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,

In [6]:
final_df = pd.DataFrame(columns= df_columns)
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [7]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ],
                                                  index = df_columns),
                                        ignore_index = True)

/tmp/ipykernel_1697/950278505.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1697/950278505.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1697/950278505.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1697/950278505.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1697/950278505.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/

In [8]:
# REMOVING LOW-MOMENTUM STOCKS AND GETTING ONLY THE TOP 30 PERFORMING STOCKS
final_df.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_df = final_df[:30]
final_df.reset_index(drop = True, inplace = True)

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)


# DOMINATED STRATEGY
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df['Ticker'])):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df['Price'][i])

final_df

90000


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.5,2.392956,36
1,DVN,64.6,1.868454,46
2,OXY,60.4,1.416552,49
3,MRO,24.6,1.403097,121
4,APA,37.95,1.10559,79
5,COP,100.3,0.961017,29
6,CF,94.08,0.854207,31
7,MCHP,65.71,0.850004,45
8,FANG,129.7,0.760275,23
9,MOS,58.09,0.7525,51


In [10]:
# SUPERIOR STRATEGY: HIGH-QUALITY MOMENTUM STOCKS "slow and steady, stable performance, less noise"
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_df = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ],
                                                  index = hqm_columns),
                                        ignore_index = True)


/tmp/ipykernel_1697/4240707639.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/tmp/ipykernel_1697/4240707639.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/tmp/ipykernel_1697/4240707639.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/tmp/ipykernel_1697/4240707639.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/tmp/ipykernel_1697/4240707639.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/tmp/ipykernel_

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

hqm_df.fillna(0,inplace=True)

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = score(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score
for time_period in time_periods:
    print(hqm_df[f'{time_period} Return Percentile'])


0      0.383234
1      0.195609
2      0.568862
3      0.870259
4      0.944112
         ...   
496    0.483034
497    0.097804
498    0.093812
499    0.479042
500    0.522954
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.187625
1      0.259481
2      0.459082
3      0.712575
4       0.96008
         ...   
496    0.518962
497     0.41517
498    0.023952
499    0.383234
500    0.235529
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.305389
1      0.586826
2      0.371257
3      0.389222
4      0.812375
         ...   
496    0.459082
497    0.694611
498     0.01996
499    0.075848
500    0.247505
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.273453
1      0.938124
2      0.279441
3       0.40519
4      0.215569
         ...   
496    0.630739
497    0.399202
498    0.071856
499    0.203593
500    0.115768
Name: One-Month Return Percentile, Length: 501, dtype: object


In [12]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [13]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:30]
hqm_df.reset_index(drop=True, inplace=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VLO,121.36,N/A,0.709373,0.976048,0.716797,0.994012,0.464972,0.998004,0.256655,0.998004,0.991517
1,DVN,64.46,N/A,1.856587,0.998004,0.637339,0.99002,0.384973,0.99002,0.121097,0.98004,0.989521
2,MRO,24.50,N/A,1.429381,0.994012,0.673414,0.992016,0.353249,0.984032,0.091187,0.954092,0.981038
3,OXY,60.38,N/A,1.444485,0.996008,0.901038,1.0,0.660055,1.0,0.052118,0.912176,0.977046
4,LB,80.82,N/A,2.357205,1.0,0.828421,0.998004,0.220129,0.958084,0.080066,0.944112,0.97505
5,MPC,92.97,N/A,0.669335,0.972056,0.499612,0.976048,0.219754,0.956088,0.108842,0.968064,0.968064
6,PXD,245.80,N/A,0.709377,0.978044,0.522947,0.982036,0.249629,0.966068,0.065202,0.932136,0.964571
7,UNM,36.66,N/A,0.281156,0.91018,0.426501,0.964072,0.285890,0.98004,0.142838,0.988024,0.960579
8,WMB,36.15,N/A,0.550231,0.962076,0.330293,0.952096,0.221857,0.96008,0.067417,0.936128,0.952595
9,APA,37.77,N/A,1.083216,0.992016,0.453614,0.974052,0.347831,0.982036,0.028773,0.858283,0.951597


In [14]:
portfolio_input()

In [15]:
position_size = float(portfolio_size) / len(hqm_df.index)
for i in range(0, len(hqm_df['Ticker'])-1):
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_df['Price'][i])
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VLO,121.36,247,0.709373,0.976048,0.716797,0.994012,0.464972,0.998004,0.256655,0.998004,0.991517
1,DVN,64.46,465,1.856587,0.998004,0.637339,0.99002,0.384973,0.99002,0.121097,0.98004,0.989521
2,MRO,24.50,1224,1.429381,0.994012,0.673414,0.992016,0.353249,0.984032,0.091187,0.954092,0.981038
3,OXY,60.38,496,1.444485,0.996008,0.901038,1.0,0.660055,1.0,0.052118,0.912176,0.977046
4,LB,80.82,371,2.357205,1.0,0.828421,0.998004,0.220129,0.958084,0.080066,0.944112,0.97505
5,MPC,92.97,322,0.669335,0.972056,0.499612,0.976048,0.219754,0.956088,0.108842,0.968064,0.968064
6,PXD,245.80,122,0.709377,0.978044,0.522947,0.982036,0.249629,0.966068,0.065202,0.932136,0.964571
7,UNM,36.66,818,0.281156,0.91018,0.426501,0.964072,0.285890,0.98004,0.142838,0.988024,0.960579
8,WMB,36.15,829,0.550231,0.962076,0.330293,0.952096,0.221857,0.96008,0.067417,0.936128,0.952595
9,APA,37.77,794,1.083216,0.992016,0.453614,0.974052,0.347831,0.982036,0.028773,0.858283,0.951597


In [16]:
# EXPORTING TO EXCEL
writer = pd.ExcelWriter('high-quality_momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='High-Quality Momentum Strategy', index=False)

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['High-Quality Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['High-Quality Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [19]:
writer.save()